In [1]:
# import torch

from transformers import BertTokenizer
from transformers import BertForSequenceClassification, AdamW, BertConfig
from transformers import get_linear_schedule_with_warmup
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
# from keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split

import pandas as pd
import numpy as np
import random
import time
import datetime


D:\ShipSupplies\DA\venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# !pip install transformers

In [3]:
# !pip install torch

In [4]:
# import tensorflow as tf

# # GPU 디바이스 목록 가져오기
# gpu_devices = tf.config.list_physical_devices('GPU')

# if len(gpu_devices) > 0:
#     print("사용 가능한 GPU가 있습니다.")
#     for device in gpu_devices:
#         print("GPU 디바이스 이름:", device.name)
# else:
#     print("사용 가능한 GPU가 없습니다.")


In [5]:
# !pip install bert-for-tf2
# !pip install tensorflow_hub

In [6]:
import tensorflow_hub as hub
from tensorflow.keras import layers
import bert

In [7]:
data = pd.read_csv('../data/raw_postpro.csv') # 오류가나면 추가해주세요 .encoding = 'cp949'
# 컬럼 삭제
df = data.drop(['청구서번호','No.',  '선박입고','완료 여부','리드타임_음수제거','청구량','견적','견적수량','견적화폐','견적단가','발주번호','발주','발주수량','발주금액','미입고 기간','리드타임','창고입고','창고입고수량','입고창고','창고출고','창고출고수량','출고선박','출고운반선','선박입고','선박입고수량','완료 여부'], axis=1)

In [8]:
delete_list = ['COMPRESSOR', 'SEAT', 'TURBINE', 'LINE', 'ANODES', 'DAMPER', 'CARD', 'BELT', 'ARM', 'SWITCH',
 'CLIP', 'BATTERY', 'ADAPTER', 'TOOL', 'CONTROL', 'BRAKE', 'TRANSFORMER', 'WINCH']
df = df[~df['key2'].isin(delete_list)]

In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 19367 entries, 0 to 20516
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Subject      19367 non-null  object
 1   Machinery    19367 non-null  object
 2   Assembly     19367 non-null  object
 3   청구품목         19367 non-null  object
 4   Part No.1    19367 non-null  object
 5   Part No.2    19367 non-null  object
 6   key1         19367 non-null  object
 7   key2         19367 non-null  object
 8   발주처          19367 non-null  object
 9   D/T          19367 non-null  object
 10  Control No.  19367 non-null  object
 11  leadtime     19367 non-null  int64 
dtypes: int64(1), object(11)
memory usage: 1.9+ MB


In [10]:
df = df[['청구품목','발주처','Machinery', 'Assembly' , "key1",'key2']]
# 'Machinery', 'Assembly', '청구품목', 'Part No.1', 'Part No.2', 'key1', '발주처'

In [11]:
from sklearn import preprocessing
label_encoders = {}  # 각 열에 대한 LabelEncoder를 저장하기 위한 딕셔너리
columns_to_encode = ['key2']  # 인코딩을 수행할 열의 이름 리스트

for column in columns_to_encode:
    le = preprocessing.LabelEncoder()
    le.fit(df[column])
    label_encoders[column] = le # 딕셔너리에 저장
    df[column+"_encoded"] = le.transform(df[column]) # 새로운 encoding 된 컬럼 추가

In [12]:
df = df.drop(['key2'], axis=1)

In [13]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 19367 entries, 0 to 20516
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   청구품목          19367 non-null  object
 1   발주처           19367 non-null  object
 2   Machinery     19367 non-null  object
 3   Assembly      19367 non-null  object
 4   key1          19367 non-null  object
 5   key2_encoded  19367 non-null  int32 
dtypes: int32(1), object(5)
memory usage: 983.5+ KB


In [14]:
# text = df[[ '청구품목', '발주처','Machinery', 'Assembly' , "key1"]].apply(lambda row: ' '.join(row), axis=1)
df_text = df[['청구품목', '발주처', 'Machinery', 'Assembly']].apply(lambda row: ' '.join(row), axis=1).to_frame(name='text')
df_text['key2'] = df['key2_encoded']

In [15]:
df_text[:20]

,text,key2
0,SEAL-O-RING-STOR HAEIN Coporation_Cheonan NO.1...,8
1,OIL COOLER & LINES HAEIN Coporation_Cheonan NO...,8
2,WASHER HAEIN Coporation_Cheonan NO.2 GENERATOR...,8
3,BOLT-HIGH TEMP HAEIN Coporation_Cheonan NO.1 G...,8
4,SEAL HAEIN Coporation_Cheonan NO.1 GENERATOR E...,8
5,CORE CHARGES FOR CYLINDER PACK AS HAEIN Copora...,8
6,PUMP GP-F TFR-REMAN HAEIN Coporation_Cheonan N...,8
7,GEAR-WTR PUMP DR HAEIN Coporation_Cheonan NO.1...,8
8,GEAR-WTR PUMP DR HAEIN Coporation_Cheonan NO.3...,8
9,GEAR-WTR PUMP DR HAEIN Coporation_Cheonan NO.3...,8


In [16]:
import re
def preprocess_text(sen):
    sentence = remove_tags(sen)
    sentence = re.sub('[^a-zA-Z]', ' ', sentence)
    sentence = re.sub(r"\s+[a-zA-Z]\s+", ' ', sentence)
    sentence = re.sub(r'\s+', ' ', sentence)
    return sentence

TAG_RE = re.compile(r'<[^>]+>')
def remove_tags(text):
    return TAG_RE.sub('', text)

en_text = []
sentences = list(df_text['text'])
for sen in sentences:
    en_text.append(preprocess_text(sen))

print(df_text.columns.values)

['text' 'key2']


In [17]:
# en_text

In [18]:
y = df_text.key2.values

In [19]:
y[:20]

array([ 8,  8,  8,  8,  8,  8,  8,  8,  8,  8,  8,  8,  8, 33,  0,  0,  0,
       14, 14, 14])

In [20]:
BertTokenizer = bert.bert_tokenization.FullTokenizer
bert_layer = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/1",
                            trainable=False)
vocabulary_file = bert_layer.resolved_object.vocab_file.asset_path.numpy()
to_lower_case = bert_layer.resolved_object.do_lower_case.numpy()
tokenizer = BertTokenizer(vocabulary_file, to_lower_case)

In [21]:
# !pip uninstall -y tensorflow

In [22]:
# !pip3 install -U "tensorflow==2.11.1"

In [23]:
import tensorflow as tf
print(tf.__version__)
print(tf.__file__)

2.10.0
D:\ShipSupplies\DA\venv\lib\site-packages\tensorflow\__init__.py


In [24]:
def tokenize_text(text):
    return tokenizer.convert_tokens_to_ids(tokenizer.tokenize(text))
tokenized_text = [tokenize_text(en) for en in en_text]

In [25]:
# en_text[0],tokenized_text[0]

In [26]:
reviews_with_len = [[text, y[i], len(text)] # 토큰화된 text, key값, text 길이
                 for i, text in enumerate(tokenized_text)]
# reviews_with_len[:5]

In [27]:
# 섞고 key2 기준으로 정렬
import random
random.shuffle(reviews_with_len)
reviews_with_len.sort(key=lambda x: x[2])
reviews_with_len[:50]

[[[7682, 2053, 13103, 3194, 13103], 0, 5],
 [[23365, 3796, 3796, 23365, 3796], 2, 5],
 [[3500, 5658, 4049, 3194, 7956, 2132], 37, 6],
 [[7744, 5658, 4049, 3194, 6718, 3847], 13, 6],
 [[13103, 7682, 2053, 13103, 3194, 13103], 0, 6],
 [[27201, 9108, 4718, 2015, 5658, 27201], 26, 6],
 [[8999, 2053, 25416, 29329, 3514, 11307], 37, 6],
 [[7744, 5658, 4049, 3194, 1055, 10216], 13, 6],
 [[6718, 3514, 11320, 2497, 3514, 5806], 22, 6],
 [[9093, 7744, 9347, 29329, 8622, 2112], 35, 6],
 [[16054, 5658, 4049, 3194, 20965, 14027], 3, 6],
 [[1051, 3614, 2364, 3194, 3514, 11307], 28, 6],
 [[10353, 5658, 4049, 3194, 7956, 2132], 9, 6],
 [[10053, 5658, 4049, 3194, 7956, 3796], 3, 6],
 [[6718, 3514, 11320, 2497, 3514, 5806], 22, 6],
 [[13103, 7682, 2053, 13103, 3194, 13103], 0, 6],
 [[5009, 5658, 4049, 3194, 7956, 2132], 13, 6],
 [[1051, 3614, 2364, 3194, 3514, 11307], 28, 6],
 [[27201, 9108, 4718, 2015, 5658, 27201], 26, 6],
 [[3608, 7682, 2373, 3796, 2373, 3796], 0, 6],
 [[21956, 11320, 2497, 3514, 580

In [28]:

sorted_text_labels = [(review_lab[0], review_lab[1]) for review_lab in reviews_with_len]
processed_dataset = tf.data.Dataset.from_generator(lambda: sorted_text_labels, output_types=(tf.int32, tf.int32))
BATCH_SIZE = 32
batched_dataset = processed_dataset.padded_batch(BATCH_SIZE, padded_shapes=((None, ), ()))
next(iter(batched_dataset))

(<tf.Tensor: shape=(32, 6), dtype=int32, numpy=
 array([[ 7682,  2053, 13103,  3194, 13103,     0],
        [23365,  3796,  3796, 23365,  3796,     0],
        [ 3500,  5658,  4049,  3194,  7956,  2132],
        [ 7744,  5658,  4049,  3194,  6718,  3847],
        [13103,  7682,  2053, 13103,  3194, 13103],
        [27201,  9108,  4718,  2015,  5658, 27201],
        [ 8999,  2053, 25416, 29329,  3514, 11307],
        [ 7744,  5658,  4049,  3194,  1055, 10216],
        [ 6718,  3514, 11320,  2497,  3514,  5806],
        [ 9093,  7744,  9347, 29329,  8622,  2112],
        [16054,  5658,  4049,  3194, 20965, 14027],
        [ 1051,  3614,  2364,  3194,  3514, 11307],
        [10353,  5658,  4049,  3194,  7956,  2132],
        [10053,  5658,  4049,  3194,  7956,  3796],
        [ 6718,  3514, 11320,  2497,  3514,  5806],
        [13103,  7682,  2053, 13103,  3194, 13103],
        [ 5009,  5658,  4049,  3194,  7956,  2132],
        [ 1051,  3614,  2364,  3194,  3514, 11307],
        [27201, 

In [29]:
import math

TOTAL_BATCHES = math.ceil(len(sorted_text_labels) / BATCH_SIZE)
TEST_BATCHES = TOTAL_BATCHES // 10
batched_dataset.shuffle(TOTAL_BATCHES)
test_data = batched_dataset.take(TEST_BATCHES)
train_data = batched_dataset.skip(TEST_BATCHES)

In [30]:
test_data

<TakeDataset element_spec=(TensorSpec(shape=(None, None), dtype=tf.int32, name=None), TensorSpec(shape=(None,), dtype=tf.int32, name=None))>

In [31]:
# for inputs, targets in train_data:
#     print(targets)


In [32]:
TOTAL_BATCHES, TEST_BATCHES

(606, 60)

In [33]:
class TEXT_MODEL(tf.keras.Model):
    def __init__(self,
                 vocabulary_size,
                 embedding_dimensions=128,
                 cnn_filters=50,
                 dnn_units=512,
                 model_output_classes=2,
                 dropout_rate=0.1,
                 training=False,
                 name="text_model"):
        super(TEXT_MODEL, self).__init__(name=name)
        self.embedding = tf.keras.layers.Embedding(vocabulary_size,
                                          embedding_dimensions)
        self.cnn_layer1 = tf.keras.layers.Conv1D(filters=cnn_filters,
                                        kernel_size=2,
                                        padding="valid",
                                        activation="relu")
        self.cnn_layer2 = tf.keras.layers.Conv1D(filters=cnn_filters,
                                        kernel_size=3,
                                        padding="valid",
                                        activation="relu")
        self.cnn_layer3 = tf.keras.layers.Conv1D(filters=cnn_filters,
                                        kernel_size=4,
                                        padding="valid",
                                        activation="relu")
        self.lstm = tf.keras.layers.LSTM(128)
        
        self.pool = tf.keras.layers.GlobalMaxPool1D()
        self.dense_1 = tf.keras.layers.Dense(units=dnn_units, activation="relu")
        self.dropout = tf.keras.layers.Dropout(rate=dropout_rate)
        if model_output_classes == 2:
            self.last_dense = tf.keras.layers.Dense(units=1,
                                           activation="sigmoid")
        else:
            self.last_dense = tf.keras.layers.Dense(units=model_output_classes,
                                           activation="softmax")
    
    def call(self, inputs, training):
        l = self.embedding(inputs)
        l_1 = self.cnn_layer1(l) 
        l_1 = self.pool(l_1) 
        l_2 = self.cnn_layer2(l) 
        l_2 = self.pool(l_2)
        l_3 = self.cnn_layer3(l)
        l_3 = self.pool(l_3)
        
        concatenated = tf.concat([l_1, l_2, l_3], axis=-1) 
        concatenated = self.dense_1(concatenated)
        concatenated = self.dropout(concatenated, training)
                # LSTM 레이어에 입력을 3D 형상으로 변환하여 전달
        lstm_output = self.lstm(tf.expand_dims(concatenated, axis=1))
        
        model_output = self.last_dense(concatenated)
        return model_output

In [34]:
VOCAB_LENGTH = len(tokenizer.vocab)
EMB_DIM = 100 #200
CNN_FILTERS = 50 #100
DNN_UNITS = 128 #256
OUTPUT_CLASSES = 43
DROPOUT_RATE = 0.1 # 0.2
NB_EPOCHS = 60
VOCAB_LENGTH
# 100 50 128 61 0.1 10 =>0.88

30522

In [35]:
text_model = TEXT_MODEL(vocabulary_size=VOCAB_LENGTH,
                        embedding_dimensions=EMB_DIM,
                        cnn_filters=CNN_FILTERS,
                        dnn_units=DNN_UNITS,
                        model_output_classes=OUTPUT_CLASSES,
                        dropout_rate=DROPOUT_RATE)

In [36]:
if OUTPUT_CLASSES == 2:
    text_model.compile(loss="binary_crossentropy",
                       optimizer="adam",
                       metrics=["accuracy"])
else:
    text_model.compile(loss="sparse_categorical_crossentropy",
                       optimizer="adam",
                       metrics=["sparse_categorical_accuracy"])

text_model.fit(train_data, epochs=NB_EPOCHS)

Epoch 1/60


546/546 [==============================] - 9s 11ms/step - loss: 2.0175 - sparse_categorical_accuracy: 0.5195
Epoch 2/60
546/546 [==============================] - 4s 7ms/step - loss: 1.0830 - sparse_categorical_accuracy: 0.7435
Epoch 3/60
546/546 [==============================] - 4s 7ms/step - loss: 0.8543 - sparse_categorical_accuracy: 0.7925
Epoch 4/60
546/546 [==============================] - 4s 7ms/step - loss: 0.7643 - sparse_categorical_accuracy: 0.8089
Epoch 5/60
546/546 [==============================] - 4s 7ms/step - loss: 0.7099 - sparse_categorical_accuracy: 0.8171
Epoch 6/60
546/546 [==============================] - 4s 7ms/step - loss: 0.6767 - sparse_categorical_accuracy: 0.8218
Epoch 7/60
546/546 [==============================] - 4s 7ms/step - loss: 0.6457 - sparse_categorical_accuracy: 0.8276
Epoch 8/60
546/546 [==============================] - 4s 7ms/step - loss: 0.6242 - sparse_categorical_accuracy: 0.8306
Epoch 9/60
546/546 [==============================] - 4s 7

In [37]:
results = text_model.evaluate(test_data)
print("60번 학습 : ", results)

60/60 [==============================] - 1s 6ms/step - loss: 0.7516 - sparse_categorical_accuracy: 0.9031
60번 학습 :  [0.7515605092048645, 0.903124988079071]


In [38]:
text_model.fit(train_data, epochs=3)

Epoch 1/3
546/546 [==============================] - 4s 7ms/step - loss: 0.4003 - sparse_categorical_accuracy: 0.8584
Epoch 2/3
546/546 [==============================] - 4s 7ms/step - loss: 0.3972 - sparse_categorical_accuracy: 0.8603
Epoch 3/3
546/546 [==============================] - 4s 7ms/step - loss: 0.4001 - sparse_categorical_accuracy: 0.8596


In [39]:
results = text_model.evaluate(test_data)
print("13번 학습 : ", results)

60/60 [==============================] - 0s 5ms/step - loss: 0.8337 - sparse_categorical_accuracy: 0.9047
13번 학습 :  [0.8336795568466187, 0.9046875238418579]


In [40]:
text_model.fit(train_data, epochs=4)

Epoch 1/4
546/546 [==============================] - 4s 7ms/step - loss: 0.3983 - sparse_categorical_accuracy: 0.8605
Epoch 2/4
546/546 [==============================] - 4s 7ms/step - loss: 0.3959 - sparse_categorical_accuracy: 0.8620
Epoch 3/4
546/546 [==============================] - 4s 7ms/step - loss: 0.3944 - sparse_categorical_accuracy: 0.8612
Epoch 4/4
546/546 [==============================] - 4s 7ms/step - loss: 0.3977 - sparse_categorical_accuracy: 0.8599


In [41]:
results = text_model.evaluate(test_data)
print("17번 학습 : ", results)

60/60 [==============================] - 0s 5ms/step - loss: 0.9170 - sparse_categorical_accuracy: 0.9078
17번 학습 :  [0.9170057773590088, 0.9078124761581421]


## pred 는각각 61개 컬럼 라벨에서의 확률

In [42]:
pred = text_model.predict(test_data)

60/60 [==============================] - 1s 4ms/step


In [43]:
pred.shape

(1920, 43)

In [44]:
pred[0]

array([9.9780184e-01, 2.4903512e-28, 8.1187972e-14, 1.7010093e-04,
       2.1845526e-07, 3.4715117e-06, 4.3863923e-14, 6.6890220e-07,
       1.1165293e-06, 3.2771923e-12, 6.6530482e-08, 5.2605945e-16,
       1.0743272e-10, 7.6955429e-04, 3.2824353e-06, 9.7387229e-08,
       4.6821755e-11, 1.3413455e-07, 2.7170498e-11, 1.2414083e-07,
       3.0519661e-19, 1.0906179e-12, 5.8234048e-19, 1.4340452e-14,
       1.0427794e-15, 2.8808728e-10, 4.2471302e-23, 1.1283725e-10,
       6.1160863e-09, 7.6672745e-15, 5.3904444e-08, 3.6062531e-08,
       3.0973821e-10, 1.2423772e-03, 5.0585383e-18, 1.9295547e-09,
       4.7341050e-08, 6.1570275e-13, 5.6546137e-06, 8.7488905e-07,
       2.4449031e-07, 1.3071990e-13, 6.6195164e-16], dtype=float32)

# predicted_result 50개의 배치(배치 사이즈 : 32) 데이터 마다 예측 라벨링을 리스트에 저장 

In [46]:
predicted_result = []
for j in range(60):
    temp=[]
    for i in range(32) :   
        predicted_class = tf.argmax(pred[i+ j*32]).numpy() ## 가장 높은 확률의 라벨링 데이터를 구함
        temp.append(predicted_class)
    predicted_result.append(temp)
        # print(predicted_class)

In [47]:
for lis in predicted_result:
    print(lis)

[0, 2, 37, 13, 0, 42, 30, 13, 22, 35, 3, 28, 9, 3, 22, 0, 13, 28, 42, 0, 22, 0, 13, 4, 22, 13, 22, 42, 40, 27, 13, 30]
[28, 0, 0, 3, 28, 4, 37, 13, 27, 22, 0, 0, 22, 40, 27, 22, 37, 27, 37, 33, 2, 41, 2, 27, 0, 34, 33, 15, 32, 34, 40, 0]
[13, 30, 40, 3, 27, 22, 40, 38, 9, 34, 32, 35, 0, 32, 9, 40, 3, 38, 2, 27, 12, 2, 32, 40, 12, 2, 27, 12, 27, 13, 3, 22]
[12, 40, 32, 13, 13, 16, 27, 13, 3, 32, 39, 37, 22, 39, 39, 3, 22, 12, 38, 12, 31, 32, 39, 0, 3, 13, 13, 13, 3, 13, 9, 12]
[28, 37, 41, 0, 41, 13, 2, 32, 22, 2, 0, 33, 13, 13, 13, 34, 3, 32, 0, 41, 0, 40, 13, 2, 27, 29, 13, 4, 3, 13, 27, 11]
[40, 3, 12, 5, 9, 12, 27, 34, 10, 13, 39, 5, 13, 13, 13, 28, 19, 41, 14, 13, 13, 29, 13, 12, 42, 0, 41, 12, 37, 12, 39, 9]
[0, 30, 13, 28, 40, 13, 13, 14, 14, 41, 28, 3, 19, 13, 12, 42, 40, 40, 28, 30, 28, 22, 13, 9, 30, 12, 3, 13, 2, 27, 38, 3]
[41, 28, 13, 3, 12, 19, 39, 34, 0, 40, 13, 42, 30, 40, 13, 4, 11, 13, 33, 15, 42, 9, 41, 12, 12, 28, 13, 13, 31, 12, 12, 28]
[42, 9, 42, 41, 30, 24, 13, 1

# true_result : 테스트 데이터에서의 실제 라벨링

In [49]:
true_result =[]
for inputs, targets in test_data.take(60):
    # 첫 번째 데이터 샘플에 대한 입력(inputs)과 라벨(targets)을 확인

    # Convert EagerTensor to numpy array
    targets_numpy = np.array(targets)

    # Convert numpy array to list
    targets_list = targets_numpy.tolist()
    true_result.append(targets_list)
    # print("Targets:", targets_list)


In [50]:
for lis in true_result:
    print(lis)

[0, 2, 37, 13, 0, 26, 37, 13, 22, 35, 3, 28, 9, 3, 22, 0, 13, 28, 26, 0, 22, 0, 40, 4, 22, 13, 22, 26, 40, 27, 40, 37]
[28, 0, 0, 3, 28, 4, 37, 13, 27, 22, 0, 0, 22, 40, 27, 22, 37, 27, 37, 33, 2, 41, 2, 27, 0, 34, 33, 15, 32, 34, 40, 0]
[13, 37, 40, 3, 27, 22, 40, 38, 9, 34, 32, 35, 0, 32, 32, 40, 3, 22, 2, 27, 12, 2, 32, 40, 12, 2, 27, 12, 27, 13, 3, 22]
[12, 40, 32, 13, 13, 16, 27, 13, 3, 32, 39, 37, 22, 39, 39, 3, 22, 12, 38, 12, 31, 32, 39, 0, 3, 13, 28, 28, 3, 13, 9, 12]
[28, 37, 41, 13, 13, 13, 2, 32, 22, 2, 0, 8, 13, 13, 28, 34, 3, 32, 0, 41, 0, 40, 13, 2, 27, 29, 13, 4, 3, 13, 27, 13]
[40, 3, 12, 8, 9, 12, 27, 34, 8, 13, 14, 8, 13, 9, 13, 28, 22, 41, 14, 13, 13, 29, 13, 12, 9, 0, 41, 12, 37, 12, 39, 9]
[0, 30, 13, 28, 40, 15, 13, 14, 14, 41, 28, 3, 22, 13, 12, 26, 40, 40, 28, 30, 28, 22, 13, 9, 30, 12, 3, 13, 2, 27, 22, 3]
[41, 28, 41, 3, 12, 22, 39, 34, 0, 40, 13, 26, 30, 40, 13, 4, 11, 13, 33, 15, 26, 9, 41, 12, 12, 28, 13, 13, 31, 12, 12, 28]
[26, 9, 26, 41, 30, 25, 13, 13,

In [51]:
from sklearn.metrics import f1_score,classification_report

In [53]:
# Calculate F1 score
f1 = f1_score([item for sublist in true_result for item in sublist],
              [item for sublist in predicted_result for item in sublist], average='macro')
print("F1 score:", f1)

F1 score: 0.8344521937028918


In [54]:
print(classification_report([item for sublist in predicted_result for item in sublist],
                            [item for sublist in true_result for item in sublist]))

              precision    recall  f1-score   support

           0       1.00      0.95      0.97       159
           2       0.95      0.88      0.91        24
           3       0.95      0.94      0.94        77
           4       1.00      1.00      1.00         9
           5       0.71      0.86      0.77        14
           6       0.80      1.00      0.89         4
           7       1.00      1.00      1.00         8
           8       0.69      0.82      0.75        11
           9       0.80      0.94      0.86        51
          10       0.92      0.94      0.93        36
          11       1.00      0.96      0.98        26
          12       0.98      0.94      0.96        62
          13       0.94      0.90      0.92       324
          14       0.89      0.98      0.93        56
          15       0.83      0.75      0.79        20
          16       0.50      1.00      0.67         2
          17       0.25      1.00      0.40         1
          18       1.00    